Import Packages

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

Create the Yelp Client

In [8]:
with open('C:/Users/ryan/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [12]:
from yelpapi import YelpAPI
yelp = YelpAPI(login['api-key'], timeout_s=5.0)
yelp

Make the file path for the json file

In [13]:
def add_results(file_name,location,term):
  try:
    os.makedirs("./data/")
  except:
    pass
  path = f"./data/{file_name}"
  try:
    with open(path,"r") as f:
      results = json.load(f)
  except:
    print("Initializing new file as empty list")
    with open(path,"w") as f:
      f.writelines(["[]"])
      results = []
  new_results = yelp.search_query(location=location,
                                  term=term,
                                  offset=len(results))
  page_amount = len(new_results["businesses"])
  num_pages = math.ceil((new_results["total"] - len(results)) / page_amount)
  for i in tqdm_notebook(range(num_pages)):
    time.sleep(0.2)
    with open(path,"w") as f:
      results.extend(new_results["businesses"])
      json.dump(results,f)
      new_results = yelp.search_query(location=location,
                                  term=term,
                                  offset=len(results))
  return results

In [14]:
len(add_results("results.json","03301","Restauraunt"))

  0%|          | 0/2 [00:00<?, ?it/s]

5

In [15]:
len(add_results("results.json","03301","Fast Food"))

  0%|          | 0/7 [00:00<?, ?it/s]

142

Export it to a dataframe

In [16]:
df = pd.read_json("./data/results.json")
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,325O-8LwokWqf2eMk6GDdg,cafe-dejavu-laconia,Cafe Dejavu,https://s3-media2.fl.yelpcdn.com/bphoto/AxRXMN...,False,https://www.yelp.com/biz/cafe-dejavu-laconia?a...,28,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 43.51969, 'longitude': -71.47842}",[delivery],$,"{'address1': '311 Court St', 'address2': '', '...",+16035247773,(603) 524-7773,32692.788372
1,1MolS8YPGNPa7VpGa9bCaw,chez-vachon-manchester-2,Chez Vachon,https://s3-media1.fl.yelpcdn.com/bphoto/ED9Po0...,False,https://www.yelp.com/biz/chez-vachon-mancheste...,182,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 42.995131, 'longitude': -71.4784426}",[delivery],$$,"{'address1': '136 Kelley St', 'address2': '', ...",+16036259660,(603) 625-9660,26706.942701
2,YGnLkxzScbYaE3aUcoe9Sw,the-friendly-toast-bedford-bedford,The Friendly Toast - Bedford,https://s3-media2.fl.yelpcdn.com/bphoto/Rax_xc...,False,https://www.yelp.com/biz/the-friendly-toast-be...,21,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 42.95311789420777, 'longitude': -...",[],$$,"{'address1': '125 S River Rd', 'address2': '',...",+16038366238,(603) 836-6238,31313.834542
3,9uq7-JcV9txvjQfdJunqlg,masa-japanese-hibachi-manchester,Masa Japanese Hibachi,https://s3-media1.fl.yelpcdn.com/bphoto/Jaslcz...,False,https://www.yelp.com/biz/masa-japanese-hibachi...,267,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 42.9511990944588, 'longitude': -7...",[delivery],$$,"{'address1': '1707 S Willow St', 'address2': '...",+16036269888,(603) 626-9888,32291.539934
4,jF60uUlmvSflv8c4zvk0iQ,fratellos-restaurants-manchester,Fratellos Restaurants,https://s3-media4.fl.yelpcdn.com/bphoto/kHHpAW...,False,https://www.yelp.com/biz/fratellos-restaurants...,227,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.0,"{'latitude': 42.99847587538801, 'longitude': -...",[delivery],$$,"{'address1': '155 Dow St', 'address2': '', 'ad...",+16036242022,(603) 624-2022,26542.546007


In [17]:
df.duplicated(subset=["id"]).sum()

0

In [18]:
df.to_csv("./data/results.csv.gz", compression="gzip", index = False)